In [ ]:
# Global Imports
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Flatten, Dropout, Input
from keras.utils import to_categorical
from tensorflow.keras.models import Model


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import joblib




In [ ]:
# Load csv data
data = pd.read_csv('midi_training.csv')

data.head()

In [ ]:
# Normilize data
scaler = MinMaxScaler()
data[['Onset_Beats', 'Duration_Beats', 'Velocity', 'Onset_Sec', 'Duration_Sec']] = scaler.fit_transform(
    data[['Onset_Beats', 'Duration_Beats', 'Velocity', 'Onset_Sec', 'Duration_Sec']]
)

joblib.dump(scaler, 'scaler.pkl')

data.head()

In [ ]:
# One-hot encode the Composer and Piece columns 
data['Composer'] = data['Composer'].astype('category')
data['Piece'] = data['Piece'].astype('category')
data['Composer'] = data['Composer'].cat.codes  # Numeric encoding
data['Piece'] = data['Piece'].cat.codes

data.head()

In [ ]:
# Create input-output pairs for sequences
X, y = [], []
sequence_length = 20  # The length of the sequence to predict the next note

for i in range(sequence_length, len(data)):
    X.append(data.iloc[i-sequence_length:i, :-2].values)  # Exclude last two columns (Composer & Piece)
    y.append(data.iloc[i, :-2].values)  # Target is the next note


In [ ]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

print(y)

In [ ]:
# Reshape X to 3D for LSTM input: [samples, time_steps, features]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])




print(X.shape)

BUILD LSTM MODEL

In [ ]:
model = Sequential()

# LSTM layer (first one)
model.add(LSTM(7, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(7))
model.add(Dropout(0.2))

# Fully connected output layer with softmax (for classification, predicting Midi_Pitch)
model.add(Dense(7, activation='softmax'))  # For Midi_Pitch prediction

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(X, y, epochs=50, batch_size=64)

In [ ]:
model.save('music_generator_model.keras')
